In [1]:
import scipy.stats as stats 
import numpy as np
import pandas as pd
import torch
import random
import os
import sys
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from tqdm import tqdm

os.chdir('..')
from hac.utils.key_points import W_LIST_POSE, W2I_POSE, \
                                 W_LIST_LEFT_HAND, W2I_LEFT_HAND, \
                                 W_LIST_RIGHT_HAND, W2I_RIGHT_HAND, \
                                 X_COLS, Y_COLS, \
                                 ALL_XY_COLS, HAND_XY_COLS, \
                                 LEFT_HAND_X_COLS, LEFT_HAND_Y_COLS, \
                                 RIGHT_HAND_X_COLS, RIGHT_HAND_Y_COLS
from hac.utils.normalizer import normalize_data, normalize_hand_data

In [2]:
os.getcwd()

'C:\\Users\\JAQQ\\YOLO\\hac'

In [3]:
torch.random.manual_seed(5566)
np.random.seed(5566)
random.seed(5566)

In [4]:
data_path = "data\\actions"
model_target = "actions"

if model_target == "actions":
    model_name = "roblox_lift_game"
    actions = ["walk", "jump", "hands_on_hips", "point_left", "point_right", "arms_lift", "punch", "trample", "lateral_raise", "stand"]
    target_columns = ALL_XY_COLS
    target_columns_x = target_columns.copy()
    target_columns += ["image_name", "label"]
elif model_target == "gestures":
    model_name = "mouse"
    actions = ["r_five", "r_zero", "l_five", "l_zero", "two_index_fingers_up", "two_index_fingers_down", "33", "55", "sit", "gesture_none"]
    target_columns = [key + "_x" for key in W_LIST_RIGHT_HAND] + [key + "_y" for key in W_LIST_RIGHT_HAND]
    target_columns += [key + "_x" for key in W_LIST_LEFT_HAND] + [key + "_y" for key in W_LIST_LEFT_HAND]
    target_columns_x = target_columns.copy()
    target_columns += ["image_name", "label"]
else:
    RunTimeError("???")

dfs = []
for idx, action in enumerate(actions):
    file_path = os.path.join(data_path, action, "data.csv")
    print(file_path)
    df = pd.read_csv(file_path)

    df.label = idx
    dfs.append(df)

data\actions\walk\data.csv
data\actions\jump\data.csv
data\actions\hands_on_hips\data.csv
data\actions\point_left\data.csv
data\actions\point_right\data.csv
data\actions\arms_lift\data.csv
data\actions\punch\data.csv
data\actions\trample\data.csv
data\actions\lateral_raise\data.csv
data\actions\stand\data.csv


In [5]:
df_train = pd.concat(dfs)
df_train = df_train.reset_index(drop=True)
df_train

,n_x,n_y,n_v,lei_x,lei_y,lei_v,le_x,le_y,le_v,leo_x,...,r_p_p_y,r_p_p_v,r_p_d_x,r_p_d_y,r_p_d_v,r_p_t_x,r_p_t_y,r_p_t_v,image_name,label
0,0.509826,0.790976,0.996212,0.524391,0.809645,0.993615,0.531443,0.805283,0.990742,0.538297,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,1626033708734.png,0
1,0.505817,0.766078,0.996582,0.519767,0.785251,0.994219,0.526347,0.781523,0.991658,0.532932,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,1626033709103.png,0
2,0.502098,0.750100,0.996911,0.515123,0.774690,0.994708,0.521026,0.772047,0.992472,0.526957,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,1626033709464.png,0
3,0.497813,0.761369,0.997201,0.508259,0.778950,0.995040,0.513120,0.775779,0.993202,0.517845,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,1626033709824.png,0
4,0.497416,0.731844,0.997474,0.509726,0.763261,0.995494,0.515646,0.761028,0.993869,0.521775,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,1626033710154.png,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4877,0.505055,0.853353,0.999973,0.514502,0.872395,0.999941,0.522862,0.870936,0.999922,0.529466,...,0.021926,0.0,0.367267,0.018427,0.0,0.371595,0.021468,0.0,1626157099102.png,9
4878,0.503939,0.853331,0.999974,0.513634,0.872416,0.999943,0.522038,0.870975,0.999925,0.528832,...,0.032285,0.0,0.364024,0.029971,0.0,0.369304,0.033603,0.0,1626157099178.png,9
4879,0.503540,0.853251,0.999975,0.513130,0.872420,0.999944,0.521526,0.870994,0.999928,0.528443,...,0.030910,0.0,0.367047,0.029005,0.0,0.371733,0.032888,0.0,1626157099270.png,9
4880,0.503719,0.853193,0.999976,0.513268,0.872423,0.999946,0.521672,0.870996,0.999930,0.528518,...,0.034011,0.0,0.360115,0.028510,0.0,0.364668,0.030136,0.0,1626157099364.png,9


In [6]:
if model_target == "actions":
    df_train = normalize_data(df_train)
    display(df_train)
if model_target == "gestures":
    df_train = normalize_hand_data(df_train)
    display(df_train)

,n_x,n_y,n_v,lei_x,lei_y,lei_v,le_x,le_y,le_v,leo_x,...,r_p_p_y,r_p_p_v,r_p_d_x,r_p_d_y,r_p_d_v,r_p_t_x,r_p_t_y,r_p_t_v,image_name,label
0,0.047294,0.654936,0.996212,0.061859,0.673605,0.993615,0.068911,0.669244,0.990742,0.075764,...,-0.136040,0.0,-0.462532,-0.136040,0.0,-0.462532,-0.136040,0.0,1626033708734.png,0
1,0.037764,0.615123,0.996582,0.051715,0.634297,0.994219,0.058295,0.630569,0.991658,0.064880,...,-0.150954,0.0,-0.468052,-0.150954,0.0,-0.468052,-0.150954,0.0,1626033709103.png,0
2,0.034514,0.583165,0.996911,0.047539,0.607755,0.994708,0.053442,0.605112,0.992472,0.059373,...,-0.166935,0.0,-0.467584,-0.166935,0.0,-0.467584,-0.166935,0.0,1626033709464.png,0
3,0.033068,0.567077,0.997201,0.043514,0.584658,0.995040,0.048375,0.581487,0.993202,0.053099,...,-0.194292,0.0,-0.464745,-0.194292,0.0,-0.464745,-0.194292,0.0,1626033709824.png,0
4,0.020204,0.547442,0.997474,0.032514,0.578859,0.995494,0.038434,0.576626,0.993869,0.044563,...,-0.184402,0.0,-0.477212,-0.184402,0.0,-0.477212,-0.184402,0.0,1626033710154.png,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4877,-0.013094,0.663125,0.999973,-0.003647,0.682167,0.999941,0.004712,0.680707,0.999922,0.011316,...,-0.168302,0.0,-0.150882,-0.171802,0.0,-0.146554,-0.168761,0.0,1626157099102.png,9
4878,-0.013794,0.664405,0.999974,-0.004100,0.683489,0.999943,0.004304,0.682049,0.999925,0.011098,...,-0.156642,0.0,-0.153710,-0.158956,0.0,-0.148430,-0.155323,0.0,1626157099178.png,9
4879,-0.014049,0.665196,0.999975,-0.004458,0.684364,0.999944,0.003937,0.682938,0.999928,0.010854,...,-0.157146,0.0,-0.150541,-0.159051,0.0,-0.145856,-0.155168,0.0,1626157099270.png,9
4880,-0.013824,0.665675,0.999976,-0.004276,0.684905,0.999946,0.004129,0.683478,0.999930,0.010974,...,-0.153507,0.0,-0.157429,-0.159008,0.0,-0.152876,-0.157382,0.0,1626157099364.png,9


In [7]:
df_train_x = df_train[target_columns_x]
df_train_y = df_train["label"]
df_train_image = df_train["image_name"]

In [8]:
class PoseGraph:
    
    num_vertices = 33
    
    def __init__(self, 
                 k_hop=3):
        
        self.k_hop = k_hop
        
        self.create_edge()
        self.create_adjacent_matrix()
        self.create_D()
        self.create_k_hop_matrix()
        
    def create_edge(self):
        self_edges = [(v, v) for v in range(self.num_vertices)]
        neighbor_edges = [(0, 1), (0, 4), (1, 2), (2, 3),
                         (3, 7), (4, 5), (5, 6), (6, 8), 
                         (9, 10), (11, 12), (11, 13), (11, 23), (12, 14),
                         (12, 24), (13, 15), (14, 16), (15, 17), (15, 19),
                         (15, 21), (16, 18), (16, 20), (16, 22), (17, 19),
                         (18, 20), (23, 24), (23, 25), (24, 26), (25, 27),
                         (26, 28), (27, 29), (27, 31), (28, 30), (28, 32), 
                         (29, 31), (30, 32), 
                        ]
        
        self.edges = self_edges + neighbor_edges
        
    def create_adjacent_matrix(self):
        self.A = np.zeros((self.num_vertices, self.num_vertices))
        for i, j in self.edges:
            self.A[i, j] = 1
            self.A[j, i] = 1
        
    def create_D(self):
        sum_row = self.A.sum(axis=1)
        self.D = np.diag(sum_row)
        
    def get_normalized_A(self):
        return np.linalg.inv(self.D) @ self.A
    
    def create_k_hop_matrix(self):
        
        As = []
        
        for hop in range(1, self.k_hop + 1):
            As.append(np.power(self.A, hop))
        
        self.A_k = np.stack(As, axis=0)
        

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class GCN(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.graph = PoseGraph()
        A_k = torch.tensor(self.graph.A_k,
                         dtype=torch.float32,
                         requires_grad=False)
        self.register_buffer('A_k', A_k)
        
        self.conv = nn.Conv2d(in_channels,
                              out_channels * self.A_k.shape[0],
                              kernel_size=(1, 1),
                              padding=(0, 0),
                              stride=(1, 1))
        
        if in_channels == out_channels:
            self.residual = lambda x: x

        else:
            self.residual = nn.Sequential(
                nn.Conv2d(
                    in_channels,
                    out_channels,
                    kernel_size=1,
                    stride=(1, 1)),
                nn.BatchNorm2d(out_channels),
            )
        
    def forward(self, x):
        
        res = self.residual(x)
        
        B, F, J, T = x.size()
        
        K = self.A_k.size()[0]
        
        x = self.conv(x)
        B, KC, J, T = x.size()
        
        x = x.view(B, K, KC // K, J, T)
        output = torch.einsum('bkcjt,kji->bcit', (x, self.A_k)) + res
        
        return output
    
class HACModel(nn.Module):
    def __init__(self, num_class):
        super().__init__()
        
        self.gcns = nn.ModuleList((
            GCN(2, 16),
            GCN(16, 32),
            GCN(32, 64),           
        ))
        
        self.fcn = nn.Conv2d(64, num_class, kernel_size=1)

    def forward(self, x):
        for gcn in self.gcns:
            x = gcn(x)
        x = F.avg_pool2d(x, x.size()[2:])
        output = self.fcn(x).squeeze()
        
        return output

In [10]:
num_class = len(actions)
model = HACModel(num_class).to('cuda')
x = torch.rand((32, 2, 33, 1)).to('cuda')
output = model(x)
print(output.size())
print(torch.argmax(output,dim=1,keepdim=True))

torch.Size([32, 10])
tensor([[9],
        [9],
        [9],
        [9],
        [9],
        [9],
        [9],
        [9],
        [9],
        [9],
        [9],
        [9],
        [9],
        [9],
        [9],
        [9],
        [9],
        [9],
        [9],
        [9],
        [9],
        [9],
        [9],
        [9],
        [9],
        [9],
        [9],
        [9],
        [9],
        [9],
        [9],
        [9]], device='cuda:0')


In [11]:
pose_cols = [key + "_x" for key in W_LIST_POSE] + [key + "_y" for key in W_LIST_POSE]

In [12]:
import torch.utils.data as data

In [13]:
class Dataset(torch.utils.data.Dataset):
    
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):

        x = self.df.loc[idx, pose_cols].values.astype('float')
        y = self.df.loc[idx, 'label']

        return x, y

In [14]:
device = 'cuda'

In [15]:
df_train['label'].values

array([0, 0, 0, ..., 9, 9, 9], dtype=int64)

In [16]:
num_class = len(actions)
model = HACModel(num_class).to('cuda')
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
dataset = Dataset(df_train)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)
for epoch in tqdm(range(0, 100)):
    epoch_loss = 0.0
    epoch_acc = 0.0
    count = 0
    for x, y in train_loader:
        x = x.view(-1, 2, 33, 1).to(device).float()
        y = y.to(device)
        
        optimizer.zero_grad()
        pred = model(x)
        loss = loss_fn(pred, y)
        
        #epoch_acc += pred
        epoch_loss += loss.item()
        loss.backward()
        
        epoch_acc += (pred.argmax(axis=1, keepdim=True).squeeze() == y).sum().item()
        optimizer.step()
        count += x.size()[0]
        
    print("epoch:", epoch_loss / len(train_loader))
    print("acc:", epoch_acc / count)

  1%|▊                                                                                 | 1/100 [00:03<05:45,  3.49s/it]

epoch: 1.795308202699898
acc: 0.5454731667349447


  2%|█▋                                                                                | 2/100 [00:06<05:42,  3.49s/it]

epoch: 0.8956977662697337
acc: 0.6845555100368701


  3%|██▍                                                                               | 3/100 [00:10<05:38,  3.49s/it]

...

epoch: 0.4941522621251399
acc: 0.8553871364195003


 65%|████████████████████████████████████████████████████▋                            | 65/100 [03:47<02:02,  3.50s/it]

epoch: 0.47983546905657826
acc: 0.869315854158132


 66%|█████████████████████████████████████████████████████▍                           | 66/100 [03:50<01:59,  3.50s/it]

epoch: 0.48164797773937773
acc: 0.860098320360508
